<a href="https://colab.research.google.com/github/marcgym2/MachineLearning/blob/main/DOE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tarea 8 (10 puntos)
- Revisa en la literatura las métricas de desempeño que se utilizan en tu problema.
- Elige al menos una métrica para aplicar a tus modelos.
- Reporta estos hallazgos en el marco teórico de tu artículo.

## Tarea 9 (10 puntos)
- Realizar un diseño de experimentos para tu problema de estudio en la que indiques los niveles y tratamientos de interés.
- Reporta en la metodología de tu artículo el diseño de experimentos que realizaste.

Los diseños factoriales completos $2^k$ son diseños experimentales que permiten evaluar el efecto de k factores, cada uno con dos niveles, sobre una o más respuestas. Estos diseños tienen las siguientes características:

Consisten en $2^k$ ensayos, uno por cada combinación posible de los niveles de los factores.
Permiten estimar los efectos principales de cada factor y las interacciones entre ellos.
Son eficientes para explorar una zona del espacio experimental y encontrar una dirección para la optimización.
Son adecuados para factores cuantitativos o cualitativos.

El resultado de un DOE de 2k es un modelo lineal que relaciona la respuesta con los factores y sus interacciones. El modelo se puede expresar como:

$y = b0 + b1x1 + b2x2 + … + bkxk + b12x1x2 + b13x1x3 + … + bk-1,kxk-1xk$

donde y es la respuesta, x1, x2, …, xk son los factores codificados en -1 y +1 según sus niveles, b0 es el término constante, b1, b2, …, bk son los coeficientes de los efectos principales, y b12, b13, …, bk-1,k son los coeficientes de las interacciones.

El modelo se puede estimar usando el método de mínimos cuadrados ordinarios (OLS), que minimiza la suma de los cuadrados de los residuos entre los valores observados y los predichos. Los coeficientes se pueden calcular usando la siguiente fórmula:

$b = (X’X)^-1 X’y$

donde X es la matriz de diseño que contiene las combinaciones de los niveles de los factores, y y es el vector de las respuestas observadas.

El modelo se puede evaluar usando diferentes medidas de ajuste, como el coeficiente de determinación (R^2), el error estándar de la estimación (SEE), el análisis de varianza (ANOVA) o los gráficos de residuos. Estas medidas permiten verificar la calidad del modelo, la significancia de los efectos y la presencia de posibles violaciones de los supuestos


In [3]:
# Importar librerías
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
# Cargar los datos
from google.colab import drive
drive.mount("/content/drive")
df_measures = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/measures_v2.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Definir las variables independientes y dependientes
x_vars = ["u_q", "coolant", "ambient", "u_d", "motor_speed", "i_q", "i_d"]
y_vars = ["pm", "torque", "stator_yoke", "stator_tooth", "stator_winding"]

In [5]:
# Crear un diseño factorial completo 2^k con k = len(x_vars)
# Se eligen dos valores diferentes para cada y_var y se realizan las combinaciones
x_levels = [-1, 1] # Niveles codificados de las variables independientes
x_comb = np.array(np.meshgrid(*[x_levels]*len(x_vars))).T.reshape(-1,len(x_vars)) # Combinaciones de los niveles
df_doe = pd.DataFrame(x_comb, columns=x_vars) 

In [6]:
# Asignar los valores reales de las variables independientes según los niveles codificados
for x in x_vars:
    df_doe[x] = df_doe[x].map({-1: df_measures[x].min(), 1: df_measures[x].max()})

In [7]:
# Generar los datos experimentales para las variables dependientes usando una muestra aleatoria del dataset original
np.random.seed(42) 
df_doe[y_vars] = df_measures[y_vars].sample(n=len(df_doe), replace=True).reset_index(drop=True)

In [8]:
# Ajustar un modelo lineal para cada variable dependiente usando el método de mínimos cuadrados ordinarios (OLS)
models = {} # Diccionario para guardar los modelos ajustados
for y in y_vars:
    formula = y + " ~ " + " + ".join(x_vars) # Fórmula del modelo
    model = ols(formula, data=df_doe).fit() # Ajustar el modelo
    models[y] = model # Guardar el modelo en el diccionario


Resumen de los modelos ajustados
- Model: Es el método de estimación de los coeficientes del modelo lineal. En este caso, es OLS (Ordinary Least Squares), que consiste en minimizar la suma de los cuadrados de los errores.
- Method: Es el algoritmo numérico que se usa para resolver el problema de optimización. En este caso, es Least Squares, que es el mismo que OLS.
- P>|t|: Es el valor p asociado al estadístico t de cada coeficiente, que mide la probabilidad de obtener un valor tan o más extremo que el observado si la hipótesis nula fuera cierta. La hipótesis nula es que el coeficiente es igual a cero.
- Covariance Type: Es el tipo de matriz de covarianza que se usa para estimar la varianza de los coeficientes. En este caso, es nonrobust, que significa que no se corrige por posibles heterocedasticidad o autocorrelación
- R-squared: Es una medida de bondad de ajuste del modelo lineal, que indica la proporción de variación de la respuesta que se explica por los factores. Su valor varía entre 0 y 1, y cuanto más cercano a 1, mejor es el ajuste del modelo. En este caso, es muy bajo.

In [9]:
# Mostrar un resumen de los modelos ajustados
for y in y_vars:
    print(f"Modelo para {y}:")
    print(models[y].summary())
    print()

Modelo para pm:
                            OLS Regression Results                            
Dep. Variable:                     pm   R-squared:                       0.070
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     1.285
Date:                Mon, 13 Mar 2023   Prob (F-statistic):              0.263
Time:                        03:26:23   Log-Likelihood:                -552.14
No. Observations:                 128   AIC:                             1120.
Df Residuals:                     120   BIC:                             1143.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      59.1380      4.685 